**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [10]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [11]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [12]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-10 15:02:07,871 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-10 15:02:07.873 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a user account into the blockchain.

In [13]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-02-10 15:02:07.922 | INFO     | __main__:<module>:13 - User Role: publisher
2025-02-10 15:02:07.923 | INFO     | __main__:<module>:24 - Creating account with name: xenodochial_morse
2025-02-10 15:02:07.924 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-10 15:02:07.933 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-10 15:02:07.934 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:02:07.935 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-10 15:02:07.935 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"


2025-02-10 15:02:07,950 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:02:07,953 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:02:08,645 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:02:08,654 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:02:08.657 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'logs/accounts.json', current total entries: 2
2025-02-10 15:02:08.658 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "create_user_account"


3 - Queries the blockchain to confirm the succesful creation of the account.

In [14]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-10 15:02:08.674 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-10 15:02:08,677 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:02:08,694 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:02:08,695 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:02:11,681 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:02:11,690 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:02:11,692 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '6a707196b743d65632f70534fc269efb3e1b711fbc8fc0f64514f14d64247643', 'timestamp': 1739199731692}
2025-02-10 15:02:11.693 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-10 15:02:11,696 - INFO - Appended data under 'xenodochial_morse@test' in logs/account_data.json
2025-02-10 15:02:11.697 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-10 

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [15]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-10 15:02:11,741 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-10 15:02:11.742 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [16]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-02-10 15:02:11.795 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-10 15:02:11.814 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-10 15:02:11.815 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-10 15:02:11.816 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-10 15:02:11.817 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-02-10 15:02:11,819 - INFO - Processing account: xenodochial_morse@test
2025-02-10 15:02:11,820 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Xenodochial Morse', 'foaf:mbox': 'xenodochial_morse@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'University of East Anglia'}, 

2025-02-10 15:02:11,869 - INFO - None
2025-02-10 15:02:11,871 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:02:14,713 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:02:14,722 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:02:14.724 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-10 15:02:14.749 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-10 15:02:14.750 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                
2025-02-10 15:02:14.752 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-02-10 15:02:14.753 | DEBUG    | integrati

6 - Queries the blockchain again to confirm the succesfull details setting.

In [17]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid, project_id = process_raw_data(details)

2025-02-10 15:02:14.773 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-10 15:02:14,776 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:02:14,791 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:02:14,793 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:02:17,744 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:02:17,752 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:02:17,755 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '121da97168c84c6dd2434d9ea0d0aede227698f48945febf0a3f534d50f75865', 'timestamp': 1739199737755}
2025-02-10 15:02:17.756 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-10 15:02:17,759 - INFO - Appended data under 'xenodochial_morse@test' in logs/account_data.json
2025-02-10 15:02:17.760 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-10 

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [18]:
user_metadata = download_json_from_ipfs(account_metadata_cid)


2025-02-10 15:02:17.813 | SUCCESS  | ipfs_functions:download_json_from_ipfs:52 - Success downloading JSON from IPFS. Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'xenodochial_morse@test', 'schema:publicKey': '697520b4385fd284f877eb6db1d312ba77fedb71ddf1cec01d2f6cb50e216ed4', 'schema:roleName': 'publisher'}, 'foaf:mbox': 'xenodochial_morse@email.com', 'foaf:name': 'Xenodochial Morse', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'University of East Anglia'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '6347-7027-2159-X'}}
